In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")

In [ ]:
modelId = 'gemini-2.5-flash'
#modelId = 'gemini-2.5-pro'

config = {
    "model": modelId,  # or other model IDs
    "temperature": 0.4,
    "max_tokens": None,
    "top_p": 0.8,
}

In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# Choose the appropriate import based on your API:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize with Google AI Studio
evaluator_llm = LangchainLLMWrapper(ChatGoogleGenerativeAI(
    model=config["model"],
    temperature=config["temperature"],
    max_tokens=config["max_tokens"],
    top_p=config["top_p"],
))

# Google AI Studio Embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

evaluator_embeddings = LangchainEmbeddingsWrapper(GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",  # Google's text embedding model
    task_type="retrieval_document"  # Optional: specify the task type
))

In [ ]:
import json
from dataset import extract_rag_data

source_data = extract_rag_data(csv_path="questions_file.csv")
dataset = [
    {
        "user_input": item["user_input"],
        "response": item["response"],
        "reference": item["reference"],
        "retrieved_contexts": item["retrieved_contexts"]
    } 
    for item in source_data
]

print(json.dumps(dataset, indent=2, ensure_ascii=False))

In [ ]:
from ragas import EvaluationDataset
eval_dataset = EvaluationDataset.from_list(dataset)
print("Features in dataset:", eval_dataset.features())
print("Total samples in dataset:", len(eval_dataset))

In [ ]:
from ragas import evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

result = evaluate(dataset=eval_dataset,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],llm=evaluator_llm)
result

In [ ]:
df = result.to_pandas()
df